# Problem2.B

In [1]:
from pyspark import SparkContext ,SparkConf

conf=SparkConf().setAppName("problem2").setMaster("local[2]")

sc = SparkContext(conf=conf)


In [2]:
points = sc.textFile('Pmax.csv').map(lambda line: (int(line.split(',')[0]), int(line.split(',')[1])))

In [3]:
def gridID(x, y):
    return int(x/20)+1+500*(500-int(y/20)-1)

def Neighbor(x):
    if x == 1:
        return [2,501,502]
    elif x == 500:
        return [499,999,1000]
    elif x == 249501:
        return[249001,249002,249502]
    elif x ==250000: 
        return[249499, 249500, 249999]
    
    elif x > 1 and x < 500:
        return [x-1, x+1, x+500, x+500-1, x+500+1]

    elif x > 249501 and x < 250000:
        return [x-1, x+1, x-500, x-500-1, x-500+1]

    elif x%500 == 1: 
        return [x+500, x+500+1, x+1, x-500, x-500+1]

    elif x%500 == 0: 
        return [x+500, x+500-1, x-1, x-500, x-500-1]
    else:
        return [x-500-1, x-500, x-500+1, x-1, x+1, x+500-1, x+500, x+500+1]


In [4]:
grids = points.map(lambda x:gridID(x[0],x[1]))

In [5]:
gridCounts=grids.map(lambda x:(x,1)).reduceByKey(lambda x,y:x+y)

In [6]:
idid=gridCounts.map(lambda x:(x[0],x[0]))
gNID= idid.flatMapValues(lambda x:Neighbor(x))
IDgN= gNID.map(lambda x:(x[1],x[0]))
jg=IDgN.join(gridCounts)
gridIDgirdCount=jg.map(lambda x:x[1]).map(lambda x:(x[0],(x[1],1)))

In [7]:
gridMean= gridIDgirdCount.reduceByKey(lambda x,y:(x[0]+y[0],x[1]+y[1])
                                     ).join(gridCounts
                                           ).map(lambda x:(x[0],x[1][1]/((x[1][0][0]+x[1][1])/(x[1][0][1]+1))))


In [8]:
IndexRanking = gridMean.sortBy(lambda x:x[1], ascending = False)

In [9]:
IndexRanking.collect()

[(155320, 1.7459016393442623),
 (205546, 1.7164948453608246),
 (137381, 1.7057356608478802),
 (100997, 1.6539379474940334),
 (217589, 1.6385224274406331),
 (25985, 1.6340782122905027),
 (9031, 1.6301020408163265),
 (14526, 1.6225352112676057),
 (1262, 1.6204986149584486),
 (25155, 1.62),
 (198197, 1.6191588785046729),
 (159507, 1.6185286103542234),
 (162013, 1.6171875),
 (233746, 1.616621983914209),
 (235671, 1.6119402985074627),
 (107889, 1.6097560975609757),
 (59850, 1.6089385474860334),
 (3579, 1.606299212598425),
 (18666, 1.6046511627906976),
 (82387, 1.601078167115903),
 (204627, 1.5985401459854016),
 (76084, 1.5975000000000001),
 (174740, 1.58560794044665),
 (64156, 1.5826771653543306),
 (139542, 1.5819477434679334),
 (2770, 1.5797872340425532),
 (38302, 1.5797872340425532),
 (99832, 1.5789473684210527),
 (171746, 1.577319587628866),
 (93319, 1.5766423357664234),
 (199511, 1.5737704918032789),
 (76566, 1.5725806451612903),
 (156479, 1.5717703349282297),
 (168483, 1.57107231920199

In [20]:
Result = IndexRanking.map(lambda x:(x[1],x[0]))
top16 = sc.parallelize(Result.top(16))
top16.collect()

[(1.7459016393442623, 155320),
 (1.7164948453608246, 205546),
 (1.7057356608478802, 137381),
 (1.6539379474940334, 100997),
 (1.6385224274406331, 217589),
 (1.6340782122905027, 25985),
 (1.6301020408163265, 9031),
 (1.6225352112676057, 14526),
 (1.6204986149584486, 1262),
 (1.62, 25155),
 (1.6191588785046729, 198197),
 (1.6185286103542234, 159507),
 (1.6171875, 162013),
 (1.616621983914209, 233746),
 (1.6119402985074627, 235671),
 (1.6097560975609757, 107889)]

# Problem2.C

In [11]:
topID = top16.map(lambda x :(x[0],x[1]))
finaltopID = topID.map(lambda x:(x[1],x[0]))
NeighborIndex = IDgN.join(finaltopID)
Result3 = NeighborIndex.groupByKey().map(lambda x:(x[0],list(x[1])))
Result3.collect()

[(14526,
  [(15025, 1.6225352112676057),
   (14525, 1.6225352112676057),
   (14025, 1.6225352112676057),
   (14026, 1.6225352112676057),
   (15026, 1.6225352112676057),
   (15027, 1.6225352112676057),
   (14527, 1.6225352112676057),
   (14027, 1.6225352112676057)]),
 (9031,
  [(9532, 1.6301020408163265),
   (8532, 1.6301020408163265),
   (9032, 1.6301020408163265),
   (9030, 1.6301020408163265),
   (9530, 1.6301020408163265),
   (8530, 1.6301020408163265),
   (9531, 1.6301020408163265),
   (8531, 1.6301020408163265)]),
 (162013,
  [(161512, 1.6171875),
   (162512, 1.6171875),
   (162012, 1.6171875),
   (161513, 1.6171875),
   (162513, 1.6171875),
   (162514, 1.6171875),
   (161514, 1.6171875),
   (162014, 1.6171875)]),
 (1262,
  [(1261, 1.6204986149584486),
   (761, 1.6204986149584486),
   (1761, 1.6204986149584486),
   (762, 1.6204986149584486),
   (1762, 1.6204986149584486),
   (1263, 1.6204986149584486),
   (763, 1.6204986149584486),
   (1763, 1.6204986149584486)]),
 (107889,
  [(10

# Problem2.D

pop-count part

In [12]:
pc0= gridCounts.map(lambda x:(x[1],x[0])).map(lambda x: (x[0],1)).reduceByKey(lambda x,y:x+y)
pc = gridCounts.map(
    lambda x:(x[1],x[0])).join(pc0).keyBy(lambda x: (x[0],x[1][1])
                                                ).map(lambda x:(x[0],x[1][1][0])
                                                     ).groupByKey().map(lambda x:(x[0],list(x[1])))
pc.collect()

[((48, 12150),
  [214416,
   41160,
   117848,
   23184,
   6268,
   12912,
   124816,
   234916,
   116884,
   119588,
   220340,
   34880,
   174516,
   135456,
   106640,
   109712,
   80128,
   83452,
   18808,
   129560,
   85240,
   181376,
   237776,
   196628,
   114648,
   26568,
   30868,
   156036,
   109056,
   139408,
   195176,
   9012,
   159212,
   64260,
   214828,
   108320,
   195832,
   3056,
   105564,
   222444,
   52204,
   88760,
   41812,
   142824,
   80268,
   116040,
   75896,
   55944,
   102864,
   52776,
   33332,
   19736,
   174300,
   178032,
   221936,
   234080,
   67056,
   56544,
   102724,
   244428,
   239512,
   182188,
   85536,
   180852,
   84868,
   194184,
   207888,
   158152,
   110844,
   216084,
   173232,
   113480,
   243988,
   147564,
   154644,
   95080,
   201528,
   268,
   184736,
   243248,
   88772,
   78444,
   174324,
   120016,
   85688,
   151232,
   46340,
   53248,
   136180,
   118156,
   31336,
   59376,
   1668,
   11

pop neighbor count part

In [21]:
neighbors = gNID.groupByKey()
pnc = neighbors.map(lambda x: list(x[1])).collect()

In [14]:
def gridID2(x,y):
    pnc1 = []
    for i in y:
        pncf  = list(list(set(x).intersection(set(i))))
        if len(pncf)>1:
            pnc1.append(pncf)
    return pnc1

In [15]:
neighbors1 = pc.map(lambda x: (x[0][0],gridID2(x[1],pnc))
    ).map(lambda x: (x[0],(len(x[1]),x[1]))
    ).keyBy(lambda x: (x[0],x[1][0])
    ).map(lambda x:(x[0],x[1][1][1]))
pncf = neighbors1.collect()

In [16]:
pncf

[((48, 13682),
  [[220947, 221948],
   [110667, 110669],
   [36783, 35783],
   [58992, 58491, 58493],
   [129203, 128703],
   [143907, 144908],
   [76892, 76893],
   [245332, 245831],
   [220339, 220341],
   [74795, 74797],
   [128475, 128975],
   [141016, 141017, 141015],
   [49968, 48969],
   [99763, 100263],
   [142427, 143428],
   [122645, 121645],
   [68633, 68631],
   [29233, 30231],
   [103211, 103213],
   [135497, 134997],
   [26941, 27439],
   [238401, 238899, 238399],
   [50216, 50217],
   [208332, 207333],
   [248505, 249004],
   [62699, 63199],
   [226652, 226151],
   [11632, 11131],
   [95648, 95147],
   [10617, 10615],
   [125803, 126303],
   [151613, 151111],
   [144144, 144145, 145144],
   [175741, 176239],
   [162192, 162193],
   [199176, 200177, 200176, 199175],
   [19036, 19037],
   [10099, 9099],
   [169337, 169335],
   [171729, 171227],
   [105065, 105063],
   [115667, 115167],
   [124153, 123652],
   [77065, 76063],
   [149369, 148868],
   [31401, 31899],
   [1536